# 📌 Résumé
## 🔍 Preprocessing

### Nous partons de notre dataframe obtenue après analyse exploratoire (création de certaines variables)

### Suppression de certaines variables qui nous sembleint non logique d'avoir dans notre modèle :

Une analyse exploratoire approfondie nous a permis d’identifier plusieurs variables dites « magiques », c’est-à-dire des variables qui contiennent des informations trop directement corrélées à la cible, voire connues uniquement après l’événement que le modèle est censé prédire.

   Plus précisément :

   - Les variables etat, etat_groupe (que nous avons créée) et client_actif véhiculaient une information quasiment équivalente à celle de la variable cible. Par exemple, il est probable que etat reflète un statut technique ou administratif du contrat, tandis que contrat représente une version métier de cet état. Leur inclusion dans le modèle aboutissait donc à une prédiction triviale.

   - De même, les variables motif_resiliation et motif_resiliation_groupe (également dérivée) ont été identifiées comme problématiques. Ces variables ne peuvent être connues qu’après la résiliation d’un contrat, et ne sont donc pas disponibles au moment où une prédiction devrait être faite en situation réelle.

Inclure ce type de variables revient à entraîner le modèle avec des informations futures, ce qui biaise gravement les résultats et donne une fausse impression de performance. Cela compromet la capacité du modèle à généraliser en production.

Pour garantir une évaluation rigoureuse et réaliste de nos performances, nous avons donc exclu ces variables du dataframe. Cette démarche assure que le modèle n’apprend qu’à partir d’informations effectivement disponibles au moment de la prédiction, conformément aux contraintes d’un usage opérationnel.

### Séparation Apprentissage/test/validation (70% / 15% / 15%)

### Valeurs manquantes : 
- Variable catégorielles : valeur missing
- Variables continues : KNN avec normalisation au préalable.

### Encodage : 

- Variable de date : 

Lors de notre analyse exploratoire nous avons crée des variables à partir des dates : 'date_debut_contrat_converted','date_dernier_mouvement_converted','date_obtention_permis_converted', 'date_mise_circulation_converted','date_naissance_converted'.

Nous allons donc seulement utiliser les variables date_debut_contrat_converted, 'anciennete_client_converted' et 'echeance_contrat_mmjj dans notre modèle.

Pour chacunes des ces variables nous acvons crée des sous variable contenant l'année, une autre le mois et une autre le jour. Afin de voir si cela allait avoir de l'importance.
Nous avons aussi créé pour chacunes des variables de cyclicité pour caoté un éventuel cycle avec cos et sin.

- Variable catégorielle a moins de 5 modalité :

Nous avons opté pour un one hot encoding. Nous aurions pour choisir le label encoding. Mais pour evité que le modèle n'inclue un quelconque ordre nous avons choisi ce dernier 

- Variables catégorielles a plus de 5 modalités : 

Nous avons opté pour un target encoder
## 🔍 Modèles utilisés

1. **XGBoost complet**

2. **Régression logistique**  
   Modèle de base simple et rapide à entraîner. Très utile comme point de comparaison.  
   - ✅ Interprétable : les coefficients indiquent l’influence de chaque variable. (utile dans un contexte metier)
   - ✅ Performant sur des données linéairement séparables.
   - ❌ Peut être limité si les relations entre variables sont non linéaires.

3. **Random Forest**  
   Ensemble d’arbres de décision créés à partir de sous-échantillons aléatoires des données.  
   - ✅ Réduit le surapprentissage grâce à l’agrégation (bagging).
   - ✅ Généralement plus robuste et performant qu’un arbre unique.
   - ✅ Moins coûteux en calcul et plus facile à paramétrer qu’un modèle comme XGBoost.
   - ❌ Moins interprétable qu’un arbre simple, mais on peut extraire l’importance des variables.

On s'attend à ce que le score soit le plus performant avec le random forest. Le modele avec XGBoost complet sert simplement de modèle de départ et d'appuie.

---

## 🎯 Objectif d'optimisation : le **F1-score**

L'objectif est de prédire quels clients vont **résilier**.
Nous avons choisi d'optimiser le **F1-Score**

- **Précision** = parmi les clients que le modèle prédit comme "à risque", combien le sont réellement ?
- **Rappel** = parmi les clients qui vont réellement résilier, combien le modèle arrive-t-il à détecter ?

Optimiser uniquement la précision :
- ➕ Moins de faux positifs.
- ➖ Mais risque de rater beaucoup de vrais résiliants.

Optimiser uniquement le rappel :
- ➕ On identifie la majorité des vrais résiliants.
- ➖ Mais on cible aussi beaucoup de clients à tort (faux positifs), ce qui coûte cher.

👉 **F1-score = compromis entre précision et rappel**  
C’est une métrique équilibrée, idéale dans notre cas où :
- Les classes sont déséquilibrées ainsi l'accuracy n'est pas réellement interpretable.
- Les **faux négatifs** (clients à risque non détectés) et les **faux positifs** (clients mal classés) ont **tous deux un impact business important**.

---

## ⚙️ Optimisation des hyperparamètres

Pour améliorer les performances des modèles, plusieurs méthodes ont été utilisées :

1. **Recherche par grille (Grid Search)**  
   - Explore toutes les combinaisons possibles de paramètres dans un espace défini.

2. **Recherche aléatoire (Random Search)**  
   - Tire au hasard des combinaisons dans l’espace des paramètres. Plus rapide que Grid Search dans de nombreux cas.

3. **Optimisation bayésienne**  
   - Utilise les résultats précédents pour choisir plus intelligemment les combinaisons suivantes.
   - Plus efficace et économe en calcul, surtout avec des modèles coûteux à entraîner.

---

## Attentes 

On s'attend à ce que le modèle le plus performant soit le modèle random forest expliquer pourquoi.

## Axes d'amelioration : 

On peut se questionner quand à la presence de certaines variables dans notre modèles. Par exemple si nous etions en entreprise il aurait ete interressant d'avoir la réelle signification de chaque variable plus en détail. Il aurait aussi été interessant d'avoir une librairie nous indiqaunt la signification de chaque modalité afin de pouvoir améliorer notre encodage. De plus avec un point de vu metier nous nous serions peut etre rendu compte qu'inclure certaines variables que nous avons inclue pourrait comporter des risque. En effet dans le cadre de l'entreprise le choix de chaque variables est cruciale. Savoir si une variable est pereine dans le temps et important : savoir si nous pouvons réellement l'inclure ou si celles ci ne risque pas de fausser le modèle avec le temps. Par exmple de type contrat ou d'offre choisie. Il se peut qu'un type de contrat ou d'offre influence notre variable cible mais est ce que cela sera le cas dans le temps. Si un nouveau contrat ou une nouvelle offre apparait est ce que cette variable se comportera toujours pareil et sera-t-elle totujours pertinente pour notre mdoele?